<a href="https://colab.research.google.com/github/mov-q/kmer-spark/blob/main/sparktest0_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import subprocess
import string
import random

KSIZE=5
BRIEF=True
# 0 significa che il file di input verrà processato interamente
BRIEF_MULTIPLIER=400
BRIEF_NOSEQ=4*BRIEF_MULTIPLIER
# id delle run da scaricare da NCBI SRA 
sraRunIDlist = ["SRR16693264", "SRR16693265"]
PREFETCH_BASEPATH="/content"
NCBI_PREFETCH_BIN="/content/sratoolkit.2.11.2-ubuntu64/bin/prefetch-orig.2.11.2"
NCBI_FASTQDUMP_BIN="/content/sratoolkit.2.11.2-ubuntu64/bin/fastq-dump-orig.2.11.2"

def generateRandomOutputSuffix(suff_length):
  temp = random.sample(string.ascii_letters,suff_length)
  return "".join(temp)

OUTPUT_SUFFIX = generateRandomOutputSuffix(4)
OUTPUT_PATH = "/content/output_"+OUTPUT_SUFFIX
print("Output path: "+OUTPUT_PATH)

Output path: /content/output_XcQT


In [2]:
exMemory = '10g'
PYSPARK_SUBMIT_ARGS = ' --driver-memory ' + exMemory + ' pyspark-shell --driver-maxResultSize 10g'
os.environ["PYSPARK_SUBMIT_ARGS"] = PYSPARK_SUBMIT_ARGS

In [3]:
!apt-get install openjdk-8-jdk-headless -qq 

In [4]:
if (os.path.exists('spark-3.2.0-bin-hadoop3.2.tgz') == False):
  !wget  https://downloads.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
  !tar xf spark-3.2.0-bin-hadoop3.2.tgz

In [5]:
if (os.path.exists('sratoolkit.2.11.2-ubuntu64.tar.gz') == False):
  !wget  https://ftp-trace.ncbi.nlm.nih.gov/sra/sdk/2.11.2/sratoolkit.2.11.2-ubuntu64.tar.gz
  !tar xf sratoolkit.2.11.2-ubuntu64.tar.gz

In [6]:
!pip install -q findspark

In [7]:
if (os.path.exists("/content/inputbrief") == False):
  !mkdir /content/inputbrief
  !wget -O brief.fastq.gz https://sra-download.ncbi.nlm.nih.gov/traces/sra57/SRZ/016777/SRR16777190/MD_2_D8.r1.fastq.gz 
  !gunzip brief.fastq.gz
if BRIEF_NOSEQ == 0:
  !cp brief.fastq /content/inputbrief/brief.fastq
else:
  !head -n $BRIEF_NOSEQ  brief.fastq > /content/inputbrief/brief.fastq


In [8]:
# setup variabili d'ambiente
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"
from pathlib import Path

import findspark
findspark.init()
# verifica delle variabili d'ambiente
findspark.find()

'/content/spark-3.2.0-bin-hadoop3.2'

In [9]:


def downloadFastq(runList):
  for singleId in runList:
    if os.path.exists(PREFETCH_BASEPATH+'/'+singleId):
      print("Prefetch delle run già completato")
    else:
      ncbiPrefetchProc = NCBI_PREFETCH_BIN + ' ' + singleId
      print("Eseguo prefetch: " + ncbiPrefetchProc)
      subprocess.check_call(ncbiPrefetchProc, shell=True)
      #ncbiFastqDump = "/content/sratoolkit.2.11.2-ubuntu64/bin/fastq-dump-orig.2.11.2 --outdir /content/input/fastq/ --gzip --skip-technical  --readids --read-filter pass --dumpbase --split-3 --clip /content/" + singleId + "/"+singleId+".sra"
      ncbiFastqDumpProc = NCBI_FASTQDUMP_BIN+" --outdir /content/input/fastq/ --skip-technical  --readids --read-filter pass --dumpbase --split-3 --clip /content/" + singleId + "/"+singleId+".sra"
      subprocess.check_call(ncbiFastqDumpProc, shell=True)

if not BRIEF:
  downloadFastq(sraRunIDlist)

In [10]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import Row
from pyspark.sql.window import Window
from pyspark.sql.functions import col, lit
import pyspark.sql.functions as func
from pyspark.sql.types import IntegerType


spark = SparkSession.builder\
        .master('local')\
        .appName('km-ark')\
        .config('spark.ui.port', '4050')\
        .config('spark.driver.maxResultSize', '10G')\
        .getOrCreate()

spark

In [11]:
if not BRIEF:
  datasetPath = Path("/content/input/fastq/")
  datasetFiles = [str(sf) for sf in datasetPath.iterdir()]
else:
  datasetPath = Path("/content/inputbrief/")
  datasetFiles = [str(sf) for sf in datasetPath.iterdir()]

print("File in input: ", len(datasetFiles))

sc = SparkContext.getOrCreate()

# read data from text file and split each line into words
#words = sc.textFile("/content/input.txt").flatMap(lambda line: line.split(" "))

def preProcess(s):
  return s

def stripFastQ(s):
  if (s[0] != '@') and (s[0] != '+'):
    return True
  else:
    return False
#rowDataFrame = sc.textFile(','.join(datasetFiles)).\
#                  map(lambda x: Row(row=stripId(x))).\
#                  zipWithIndex().\
#                  toDF(["fastqRow","index"])

# per ritornare tuple:    map(lambda x: (preProcess(x),1)).\
rawData = None

rawData = sc.textFile(','.join(datasetFiles)).zipWithIndex()
#rawData.collect()


File in input:  1


In [12]:


#filtered = rowData.filter(lambda line: stripFastQ(line)).collect()
rawDataFrame = rawData.map(lambda line: preProcess(line)).toDF(["fastqRow","index"])
rawDataFrame.show(truncate=False)
# aggiungiamo una chiave per il raggruppamento in 4 righe
gSeq = rawDataFrame.withColumn("group", col("index")/4)
gSeq = gSeq.withColumn("group", col("group").cast(IntegerType()))
# mostriamo il dataframe risultante
gSeq.show(truncate=False)
gSeqRDD = gSeq.rdd
#gSeqRDD.collect()

+-------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|fastqRow                                                                                                                                               |index|
+-------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|@A00152:72:HFKKWDSXX:1:1101:24632:1000 1:N:0:0                                                                                                         |0    |
|CTTATAAGCTCAATGGTGTTTTCCACCGTGATCCGCCGCTGACGCAGCGTGCTCTCAAATATCTGCCGGAAGACACATTGCGGTTCGTTGATAATAAAGCTACAGGCGTTATGTCTTCCCGGCTCAGTAAAATCGACATCTGCAATTTGCG|1    |
|+                                                                                                                                                      |2    |
|,FFFFFFF:FFFFF,FFFFFFFFFFFFFFF:FFFFFFFF

In [13]:
gSeqRDDgroup = gSeqRDD.map(lambda line: (line.group, (line.fastqRow, line.index)))
#gSeqRDDgroup.collect()

In [14]:
seqGroups = gSeqRDDgroup.groupByKey()

def setScoreDict():

  valString = "!\"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz{|}~"
  valDict = {}
  idx = 0
  for symbol in valString:
    valDict[symbol] = len(valString)-idx
    idx = idx+1

  return valDict
  
scoreTable = setScoreDict()

def parseSequenceId(sh):
  if (sh[0] == '@'):
    return sh.split()[0][1:]
       

def seqQuality(score_table, st):
  acc = 0
  for c in st:
    acc = acc + score_table[c]
  return acc

# seqCleaner per il momento implementa un metodo piuttosto rozzo per valutare la
# qualità dell'
def seqProcessor(rowGroup, st):
  score = 0  
  groupNumber = rowGroup[0]
  rowList = list(rowGroup[1])
  # ora abbiamo una lista di 4 elementi, il gruppo di righe
  seqHeader = rowList[0][0]
  rawSequence = rowList[1][0]
  seqScore = rowList[3][0]
  score = (seqQuality(scoreTable, rawSequence) / len(rawSequence))

  seqId = parseSequenceId(seqHeader)

  if (score > 54):
    return (groupNumber,(rawSequence, seqId))

cleanedSeq = seqGroups.map(lambda r: seqProcessor(r,scoreTable))
cleanedSeq = cleanedSeq.filter(lambda x: x != None)
#cleanedSeq.collect()

In [15]:
# Otteniamo un RDD delle sole sequenze filtrate al passaggio precedente
#seqRdd = cleanedSeq.map(lambda r: r[1][0])
seqRdd = cleanedSeq.map(lambda r: (r[1][0],r[1][1]))
#seqRdd.collect()

In [16]:
# Calcoliamo i k-mer di lunghezza KSIZE dalle sequenze
# Per ogni sequenza otteniamo una lista di k-mer che rendiamo in una sola string
def singleSeqKmer(l, ksize):
  sequence = l.strip()
  kmers = []

  for i in range(0,len(l)-ksize):
    kmers.append(sequence[i:i+ksize+1])

  kmerString = ' '.join(kmers)
  return kmerString

fullKmerSeq = seqRdd.map(lambda kmPair: (singleSeqKmer(kmPair[0],KSIZE),kmPair[1]))
#fullKmerSeq.collect()

In [17]:
# Dalla singola stringa andiamo ad effettuarle lo split finale
#kmerSplit = fullKmerSeq.map(lambda line: (line[0].split(" "),line[1]))
kmerSplit = fullKmerSeq.flatMap(lambda line: [(x, line[1]) for x in line[0].split(" ")])

#kmerSplit.collect()


In [18]:
def occReducer(lAccu, lCur):
  return (lAccu[0]+lCur[0],lAccu[1]+lCur[1])

kmerCounts = kmerSplit.map(lambda singleKmer: (singleKmer[0], (1,[singleKmer[1]]))).reduceByKey(lambda accu, cur: occReducer(accu, cur))
kmerCounts.saveAsTextFile(OUTPUT_PATH)

# stampiamo le prime 20 entries del RDD
kmerCounts.take(20)

[('CTTATA',
  (5,
   ['A00152:72:HFKKWDSXX:1:1101:24632:1000',
    'A00152:72:HFKKWDSXX:1:1101:11044:2769',
    'A00152:72:HFKKWDSXX:1:1101:6433:3051',
    'A00152:72:HFKKWDSXX:1:1101:24795:3380',
    'A00152:72:HFKKWDSXX:1:1101:9335:4382'])),
 ('TTATAA',
  (11,
   ['A00152:72:HFKKWDSXX:1:1101:24632:1000',
    'A00152:72:HFKKWDSXX:1:1101:8820:2707',
    'A00152:72:HFKKWDSXX:1:1101:11044:2769',
    'A00152:72:HFKKWDSXX:1:1101:18096:3145',
    'A00152:72:HFKKWDSXX:1:1101:23619:3255',
    'A00152:72:HFKKWDSXX:1:1101:23619:3255',
    'A00152:72:HFKKWDSXX:1:1101:24795:3380',
    'A00152:72:HFKKWDSXX:1:1101:25599:4210',
    'A00152:72:HFKKWDSXX:1:1101:30897:4272',
    'A00152:72:HFKKWDSXX:1:1101:14805:4304',
    'A00152:72:HFKKWDSXX:1:1101:9335:4382'])),
 ('TATAAG',
  (5,
   ['A00152:72:HFKKWDSXX:1:1101:24632:1000',
    'A00152:72:HFKKWDSXX:1:1101:29803:1877',
    'A00152:72:HFKKWDSXX:1:1101:11044:2769',
    'A00152:72:HFKKWDSXX:1:1101:2112:4053',
    'A00152:72:HFKKWDSXX:1:1101:14805:4304']